In [1]:
import pyaudio
import pyttsx3
from pydub import AudioSegment
from gtts import gTTS
import speech_recognition as sr
from googletrans import Translator
import json
import random
import re

In [6]:
class MarinaAssistant:
    def __init__(self):
        self.recognizer = sr.Recognizer()
        self.synthesizer = pyttsx3.init()
    
    def speech_to_text(self):
        # распознаем речь
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Скажите что-нибудь...")
            audio = r.listen(source)
        try:
            text = r.recognize_google(audio, language="ru-RU")
            print(text)
            return text
        except sr.UnknownValueError:
            print("Ошибка распознавания речи")
            return ""
        except sr.RequestError:
            print("Ошибка соединения с сервером распознавания речи")
            return ""
      
    def speak(self, text):
        # Вывод голоса Марины   
        engine = pyttsx3.init()
        # Скорость и громкость
        engine.setProperty('rate', 150)  # 150 words per minute
        engine.setProperty('volume', 0.8)  # 80% volume
        # смена голоса в зависимости от языка
        voices = engine.getProperty('voices')
        in_rus = bool(re.search('[а-яА-Я]', text))
        if in_rus:
            engine.setProperty('voice', voices[0].id) 
        if in_rus == False:
            engine.setProperty("voice", voices[1].id)

        engine.say(text)
        engine.runAndWait()


    def create_file(self):
        # Файл с фразами встроенный
        phrases = ["I’m from Russia", "I love you", "My cat is fat", "This house is red", "I go to school"]
        with open("phrases.txt", "w") as file:
            for phrase in phrases:
                file.write(phrase + "\n")
        
    
    def add_phrase(self):
        self.speak("Хорошо! Введите фразу")
        # Добавить пользовательскую фразу в базу данных
        phrase = input("Введите фразу: ")
        with open("phrases.txt", "a") as file:
            file.write(phrase + "\n")
        print("Добавлено!")

    
    def practice(self):
        # Команда практика
        self.speak("Прочитайте текст")
        with open("phrases.txt", "r") as file:
            phrases = file.readlines()
        random_phrase = random.choice(phrases)
        print("Прочитайте текст:")
        print(random_phrase)
         
        original_text = random_phrase
        recognized_text = self.speech_to_text()
        print(f"Вы сказали: {recognized_text}")
        self.compare_text(original_text, recognized_text)

    
    def compare_text(self, original_text, recognized_text):
        # Сравниваем речь пользователя с оригиналом
        punctuations = ".?!,:;’-–—”()[]{}<>“/…*&#~\@^|«»'"
        original_no_punc = ""
        original_text = original_text.strip()
        for char in original_text.lower():
            if char not in punctuations:
                original_no_punc += char
        recognized_no_punc = ""
        recognized_text = recognized_text.strip()
        for char in recognized_text.lower():
            if char not in punctuations:
                recognized_no_punc += char
    
        if original_no_punc == recognized_no_punc:
            print("Правильно!")
            self.speak("Правильно!")
        else:
            print("Неправильно")
            self.speak("Неправильно, потренируемся ещё?")
            more_practice = self.speech_to_text()
            if more_practice.lower().strip() == "да":
                self.practice()

                
    def translate_text(self, text):
        # Определяем язык
        in_rus = bool(re.search('[а-яА-Я]', text))
        if in_rus:
            d = 'en'
        if in_rus == False:
            d = 'ru'
        # Берем полученное сообщение и переводим его
        translator = Translator()
        translated_text = translator.translate(text, dest=d)
        return translated_text.text


    def run(self):
        # команды
        while True:
            self.create_file()
            command = self.speech_to_text().lower().strip()

            if command == "марина переведи":
                self.speak("Введите текст для перевода")
                text = input("Введите текст для перевода: ")
                translation = self.translate_text(text)
                print(f"Перевод: {translation}")
                self.speak("Зачитать перевод вслух?")
                play_audio = self.speech_to_text()
                if play_audio.lower().strip() == "да":
                    self.speak(translation)

            elif command == "марина зачитай вслух":
                self.speak("Введите текст для озвучивания")
                text = input("Введите текст для озвучивания: ")
                self.speak(text)
            elif command == "марина практика":
                self.practice()
            elif command == "марина добавь фразу":
                self.add_phrase()
            elif command == "марина пока":
                self.speak("Пока!")
                print('Пока!')
                break
            else:
                print("Неизвестная команда.")


assistant = MarinaAssistant()
assistant.run()

Скажите что-нибудь...
Марина практика
Прочитайте текст:
My cat is fat

Скажите что-нибудь...
My Cat is Fat
Вы сказали: My Cat is Fat
Правильно!
Скажите что-нибудь...
Марина Переведи


Введите текст для перевода:  Параход приближался к берегу


Перевод: The pair of paradise was approaching the shore
Скажите что-нибудь...
да
Скажите что-нибудь...
Марина Зачитай вслух


Введите текст для озвучивания:  This book is mine


Скажите что-нибудь...
Марина Добавь фразу


Введите фразу:  Happy birthday


Добавлено!
Скажите что-нибудь...
Марина пока
Пока!


In [30]:
engine = pyttsx3.init()
voices = engine.getProperty('voices')
for voice in voices:
    print(f"Voice: {voice.name}")

Voice: Microsoft Irina Desktop - Russian
Voice: Microsoft Zira Desktop - English (United States)
Voice: Microsoft David Desktop - English (United States)
